# Lesson 6: Apache Spark & PySpark for ML Data Processing

**Module 3: Data & Pipeline Engineering** | **Time**: 4-5 hours | **Difficulty**: Intermediate-Advanced

---

## 🎯 Learning Objectives

✅ Understand why single-machine processing fails at scale  
✅ Learn Spark’s distributed architecture (Driver, Executors, DAG)  
✅ Master PySpark DataFrame operations for ML data prep  
✅ Understand partitioning, caching, and optimization  
✅ Answer 5 interview questions on Spark and distributed computing  

---

## 📚 Table of Contents

1. [Why Spark? The Scale Problem](#1-why-spark)
2. [Spark Architecture](#2-architecture)
3. [Spark Execution Model: Lazy Evaluation + DAG](#3-execution)
4. [PySpark DataFrame Basics](#4-pyspark-basics)
5. [Data Transformations for ML](#5-transformations)
6. [Partitioning & Optimization](#6-partitioning)
7. [Spark vs Pandas Comparison](#7-spark-vs-pandas)
8. [Exercises](#8-exercises)
9. [Interview Preparation](#9-interview)

---

## 1. Why Spark? The Scale Problem <a id='1-why-spark'></a>

Pandas is brilliant for datasets that fit in memory on a single machine. But what happens when they don’t?

### The Scale Wall

```
  Dataset Size vs Processing Approach:

  < 1 GB     ──▶  Pandas on laptop         ✅ Easy
  1-10 GB    ──▶  Pandas with chunking     ⚠️ Painful
  10-100 GB  ──▶  Pandas fails             ❌ Out of memory
  100+ GB    ──▶  Need distributed system  🚀 Spark!
  1+ TB      ──▶  Spark required           🚀🚀 Spark!
```

### Pandas vs Spark Under the Hood

```
  PANDAS (Single Machine):
  ┌────────────────────────────┐
  │      Your Laptop (32 GB)      │
  │  ┌───────────────────────┐  │
  │  │ ENTIRE DataFrame in   │  │
  │  │ RAM at once           │  │
  │  └───────────────────────┘  │
  │  100 GB dataset? OOM! 💥      │
  └────────────────────────────┘

  SPARK (Distributed Cluster):
  ┌─────────┐ ┌─────────┐ ┌─────────┐ ┌─────────┐
  │ Worker1 │ │ Worker2 │ │ Worker3 │ │ Worker4 │
  │ 25 GB   │ │ 25 GB   │ │ 25 GB   │ │ 25 GB   │
  └─────────┘ └─────────┘ └─────────┘ └─────────┘
       100 GB split across 4 workers ✅
       Each processes its chunk in parallel!
```

---

## 2. Spark Architecture <a id='2-architecture'></a>

### Core Components

```
  ┌────────────────────────────────────────────────────────┐
  │                    SPARK APPLICATION                     │
  ├────────────────────────────────────────────────────────┤
  │  DRIVER (your program)                                   │
  │  ┌────────────────────────────────────────────────┐  │
  │  │ SparkContext → Creates DAG → Submits Jobs           │  │
  │  └────────────────────────────────────────────────┘  │
  │                         │                                │
  │              ┌──────────┴──────────┐                  │
  │  CLUSTER    │  CLUSTER MANAGER   │                  │
  │  MANAGER    │  (YARN/Mesos/K8s)  │                  │
  │              └──────────┬──────────┘                  │
  │                         │                                │
  │         ┌─────────────┼─────────────┐              │
  │  ┌──────────┐  ┌──────────┐  ┌──────────┐      │
  │  │ Executor │  │ Executor │  │ Executor │      │
  │  │ [Task]   │  │ [Task]   │  │ [Task]   │      │
  │  │ [Task]   │  │ [Task]   │  │ [Task]   │      │
  │  └──────────┘  └──────────┘  └──────────┘      │
  │     Worker 1       Worker 2       Worker 3       │
  └────────────────────────────────────────────────────────┘
```

| Component | Role |
|-----------|------|
| **Driver** | Your PySpark program. Creates the SparkContext, builds the DAG of operations, submits jobs |
| **Cluster Manager** | Allocates resources (YARN, Mesos, Kubernetes, or Standalone) |
| **Executors** | JVM processes on worker nodes. Execute tasks and store data in memory/disk |
| **Tasks** | Individual unit of work. One task per partition per stage |

---

## 3. Spark Execution Model: Lazy Evaluation + DAG <a id='3-execution'></a>

Spark uses **lazy evaluation**: transformations are NOT executed immediately. Instead, Spark builds a **DAG (Directed Acyclic Graph)** of operations. Execution only happens when an **action** is called.

### Transformations vs Actions

```
  TRANSFORMATIONS (Lazy - build the plan):
  .filter()  .select()  .groupBy()  .join()  .withColumn()
        │         │          │         │          │
        └────────┴─────────┴────────┴─────────┘
                        │
                   Build DAG
                   (no execution yet!)
                        │
  ACTIONS (Trigger execution):
  .show()  .collect()  .count()  .write()  .toPandas()
        │
    NOW the DAG executes!
    Spark optimizes the entire plan first.
```

### Why Lazy Evaluation?

```
  Example: df.filter(col('age') > 30).select('name', 'age')

  Eager (like Pandas):              Lazy (Spark):
  1. Load ALL columns               1. Build plan
  2. Filter rows                    2. Optimize: push filter down
  3. Select 2 columns               3. Execute: read only 2 columns
  4. Wasted: loaded unused columns      + filter in one pass

  Lazy is SMARTER — it sees the whole plan and optimizes!
```

---

## 4. PySpark DataFrame Basics <a id='4-pyspark-basics'></a>

PySpark DataFrames look similar to Pandas but run on a distributed cluster.

In [ ]:
# ============================================================
# PySpark Setup (local mode for learning)
# In production, Spark runs on a cluster (YARN, K8s, Databricks)
# ============================================================
try:
    from pyspark.sql import SparkSession
    from pyspark.sql import functions as F
    from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
    
    spark = SparkSession.builder \
        .master('local[*]') \
        .appName('Module3_Lesson6') \
        .getOrCreate()
    
    spark.sparkContext.setLogLevel('ERROR')
    print(f"✅ Spark {spark.version} initialized (local mode)")
    print(f"   Cores: {spark.sparkContext.defaultParallelism}")
    SPARK_AVAILABLE = True
except ImportError:
    print("⚠️ PySpark not installed. Run: pip install pyspark")
    print("   The code examples below show what would execute.")
    SPARK_AVAILABLE = False

In [ ]:
# ============================================================
# Creating DataFrames in PySpark
# ============================================================
import pandas as pd
import numpy as np

# Generate synthetic ML dataset
np.random.seed(42)
n = 100_000

pandas_df = pd.DataFrame({
    'user_id': np.arange(n),
    'category': np.random.choice(['electronics', 'clothing', 'food', 'books'], n),
    'price': np.round(np.random.uniform(5, 500, n), 2),
    'quantity': np.random.randint(1, 10, n),
    'rating': np.round(np.random.uniform(1, 5, n), 1),
})

if SPARK_AVAILABLE:
    # Convert Pandas to Spark DataFrame
    sdf = spark.createDataFrame(pandas_df)
    
    # Basic operations
    print("Schema:")
    sdf.printSchema()
    
    print(f"\nPartitions: {sdf.rdd.getNumPartitions()}")
    print(f"Row count: {sdf.count():,}")
    
    sdf.show(5)
else:
    print("[Without Spark] Pandas DataFrame:")
    print(pandas_df.head())

In [ ]:
# ============================================================
# Common PySpark Operations (Pandas equivalents shown)
# ============================================================
if SPARK_AVAILABLE:
    # SELECT columns
    print("1. SELECT columns:")
    sdf.select('category', 'price').show(3)
    
    # FILTER rows
    print("2. FILTER rows:")
    sdf.filter(F.col('price') > 400).show(3)
    
    # ADD new column
    print("3. ADD computed column:")
    sdf.withColumn('total', F.col('price') * F.col('quantity')).show(3)
    
    # GROUP BY + aggregate
    print("4. GROUP BY + AGGREGATE:")
    sdf.groupBy('category').agg(
        F.avg('price').alias('avg_price'),
        F.sum('quantity').alias('total_qty'),
        F.count('*').alias('count')
    ).show()
else:
    print("PySpark equivalents of common Pandas operations:")
    print("""  
    | Pandas                          | PySpark                                  |
    |---------------------------------|------------------------------------------|
    | df[['cat', 'price']]            | sdf.select('cat', 'price')               |
    | df[df.price > 400]              | sdf.filter(F.col('price') > 400)         |
    | df['total'] = df.price * df.qty | sdf.withColumn('total', col*col)         |
    | df.groupby('cat').agg(...)       | sdf.groupBy('cat').agg(F.avg('price'))   |
    """)

## 5. Data Transformations for ML <a id='5-transformations'></a>

### Common ML Feature Engineering in PySpark

```
  Raw Data ─▶ Feature Engineering Pipeline in Spark:

  ┌──────────┐  ┌──────────┐  ┌──────────┐  ┌──────────┐  ┌─────────┐
  │ Handle   │▶│ Encode   │▶│ Window   │▶│ Scale    │▶│ Save as │
  │ Nulls    │  │ Categor. │  │ Features │  │ Numeric  │  │ Parquet │
  └──────────┘  └──────────┘  └──────────┘  └──────────┘  └─────────┘
```

In [ ]:
# ============================================================
# ML Feature Engineering in PySpark
# ============================================================
if SPARK_AVAILABLE:
    from pyspark.sql.window import Window
    
    # 1. Handle nulls
    sdf_clean = sdf.fillna({'rating': 3.0, 'price': 0})
    
    # 2. One-hot encode categoricals (StringIndexer + OneHotEncoder)
    from pyspark.ml.feature import StringIndexer, OneHotEncoder
    indexer = StringIndexer(inputCol='category', outputCol='category_idx')
    sdf_indexed = indexer.fit(sdf_clean).transform(sdf_clean)
    
    # 3. Window features (rolling aggregates)
    window = Window.partitionBy('category').orderBy('user_id').rowsBetween(-10, 0)
    sdf_features = sdf_indexed.withColumn(
        'rolling_avg_price', F.avg('price').over(window)
    )
    
    # 4. Interaction features
    sdf_features = sdf_features.withColumn(
        'price_x_qty', F.col('price') * F.col('quantity')
    )
    
    # 5. Binning
    sdf_features = sdf_features.withColumn(
        'price_bucket',
        F.when(F.col('price') < 50, 'low')
         .when(F.col('price') < 200, 'medium')
         .otherwise('high')
    )
    
    print("Engineered features:")
    sdf_features.select('category', 'price', 'category_idx', 
                        'rolling_avg_price', 'price_x_qty', 'price_bucket').show(5)
else:
    print("PySpark ML feature engineering patterns:")
    print("1. fillna() - Handle missing values")
    print("2. StringIndexer + OneHotEncoder - Encode categoricals")
    print("3. Window functions - Rolling aggregates")
    print("4. withColumn() - Interaction features")
    print("5. when/otherwise - Binning")

## 6. Partitioning & Optimization <a id='6-partitioning'></a>

### What Is a Partition?

```
  A Spark DataFrame is split into PARTITIONS:
  Each partition is processed by ONE task on ONE core.

  ┌───────────────────────────────────────────┐
  │              DataFrame (100M rows)            │
  ├──────────┬──────────┬──────────┬──────────┤
  │ Part 0   │ Part 1   │ Part 2   │ Part 3   │
  │ 25M rows │ 25M rows │ 25M rows │ 25M rows │
  └──────────┴──────────┴──────────┴──────────┘
    Core 1     Core 2     Core 3     Core 4
    (parallel execution)

  Rule of thumb: 2-4 partitions per CPU core
  Too few partitions:  underutilized cores
  Too many partitions: scheduling overhead
```

### Narrow vs Wide Transformations

```
  NARROW (no shuffle):          WIDE (requires shuffle):
  filter, select, map           groupBy, join, repartition

  ┌────┐  ┌────┐  ┌────┐       ┌────┐  ┌────┐  ┌────┐
  │ P0 │  │ P1 │  │ P2 │       │ P0 │  │ P1 │  │ P2 │
  └─┬──┘  └─┬──┘  └─┬──┘       └─┬──┘  └─┬──┘  └─┬──┘
    │       │       │             │ ╲  │ /  │
    │       │       │             │  ╲│/   │    SHUFFLE!
  ┌─┴──┐  ┌─┴──┐  ┌─┴──┐       ┌─┴──┐  ┌─┴──┐  ┌─┴──┐
  │ P0 │  │ P1 │  │ P2 │       │ P0 │  │ P1 │  │ P2 │
  └────┘  └────┘  └────┘       └────┘  └────┘  └────┘
  Fast! No data movement.       Expensive! Data moves across network.
```

### Key Optimization Tips

| Tip | Description |
|-----|-------------|
| **Minimize shuffles** | Avoid unnecessary groupBy/join operations |
| **Cache wisely** | Cache DataFrames you reuse: `sdf.cache()` |
| **Use Parquet** | Enables predicate pushdown and column pruning |
| **Broadcast small tables** | For joins with small lookup tables |
| **Partition by** | When writing: `sdf.write.partitionBy('date')` |

---

In [ ]:
# ============================================================
# Optimization Demo: Caching and Partition Management
# ============================================================
if SPARK_AVAILABLE:
    import time
    
    # Without caching
    start = time.time()
    sdf.groupBy('category').agg(F.avg('price')).collect()
    sdf.groupBy('category').agg(F.sum('quantity')).collect()
    no_cache = time.time() - start
    
    # With caching
    sdf.cache()
    sdf.count()  # Trigger caching
    
    start = time.time()
    sdf.groupBy('category').agg(F.avg('price')).collect()
    sdf.groupBy('category').agg(F.sum('quantity')).collect()
    with_cache = time.time() - start
    
    sdf.unpersist()
    
    print(f"Without caching: {no_cache:.3f}s")
    print(f"With caching:    {with_cache:.3f}s")
    print(f"Speedup: {no_cache/with_cache:.1f}x")
else:
    print("Caching keeps data in memory between operations.")
    print("Use .cache() when you reuse a DataFrame multiple times.")

## 7. Spark vs Pandas Comparison <a id='7-spark-vs-pandas'></a>

### API Comparison

| Operation | Pandas | PySpark |
|-----------|--------|----------|
| Read CSV | `pd.read_csv('f.csv')` | `spark.read.csv('f.csv')` |
| Read Parquet | `pd.read_parquet('f.pq')` | `spark.read.parquet('f.pq')` |
| Select cols | `df[['a', 'b']]` | `sdf.select('a', 'b')` |
| Filter | `df[df.a > 5]` | `sdf.filter(F.col('a') > 5)` |
| New column | `df['c'] = df.a * 2` | `sdf.withColumn('c', F.col('a') * 2)` |
| GroupBy | `df.groupby('a').mean()` | `sdf.groupBy('a').agg(F.avg('b'))` |
| Sort | `df.sort_values('a')` | `sdf.orderBy('a')` |
| Join | `pd.merge(df1, df2, on='id')` | `sdf1.join(sdf2, 'id')` |
| Count | `len(df)` | `sdf.count()` |

### When to Use Which

```
  Data fits in RAM?          ── YES ─▶  Pandas (faster for small data)
         │
        NO
         │
  Need real-time processing? ── YES ─▶  Spark Streaming
         │
        NO
         │
  Batch processing at scale  ─────▶  Spark (batch)
```

---

## 8. Exercises <a id='8-exercises'></a>

### Exercise 1: PySpark Feature Engineering
Using PySpark, create a feature engineering pipeline that: handles nulls, creates a log-transformed price column, creates a price/quantity ratio, and saves as partitioned Parquet.

### Exercise 2: Optimization
Given a large DataFrame with 50 columns, you only need 5 for your ML model. Write code that reads efficiently (column pruning) and caches for repeated use.

### Exercise 3: Broadcast Join
Create a small lookup table (category → department mapping) and demonstrate a broadcast join with a large transactions DataFrame. Compare performance with a regular join.

---

## 9. Interview Preparation <a id='9-interview'></a>

### Q1: "Explain Spark’s lazy evaluation. Why does it matter?"

**Answer:**  
"Spark doesn't execute transformations immediately. Instead it builds a DAG (Directed Acyclic Graph) of operations. Execution only happens when an action like `.count()` or `.collect()` is called.

This matters because Spark can **optimize the entire plan** before executing. For example, if I select 3 columns then filter rows, Spark pushes the filter down to read less data. Without lazy evaluation, it would read all data first, then filter — wasting I/O."

---

### Q2: "What’s a shuffle in Spark? Why is it expensive?"

**Answer:**  
"A shuffle is a redistribution of data across partitions. It happens during wide transformations like `groupBy`, `join`, and `repartition`.

It’s expensive because:
1. Data must be **serialized**, sent **over the network**, and **deserialized**
2. Intermediate data is written to **disk** for fault tolerance
3. It creates a **stage boundary** in the DAG

To minimize shuffles: use broadcast joins for small tables, pre-partition data by join keys, and avoid unnecessary `repartition()` calls."

---

### Q3: "How would you optimize a Spark job that’s running slowly?"

**Answer:**  
"My debugging checklist:
1. **Check the Spark UI** for stage timelines, shuffle sizes, and skewed partitions
2. **Data skew**: One partition much larger than others? Use salting or repartition
3. **Shuffle reduction**: Can I broadcast smaller DataFrames in joins?
4. **Partition count**: Too few = underutilized cores. Too many = overhead. Aim for 2-4 per core
5. **Caching**: Am I re-computing the same DataFrame? Cache it
6. **Predicate pushdown**: Am I filtering early? Read Parquet with column pruning
7. **Serialization**: Use Kryo over Java serialization for speed"

---

### Q4: "Explain narrow vs wide transformations."

**Answer:**  
"**Narrow** transformations (filter, select, map) operate on data within the same partition — no data movement needed. They’re fast and don’t require a shuffle.

**Wide** transformations (groupBy, join, distinct) require data to move between partitions (shuffle). They create stage boundaries and are the primary source of Spark performance issues.

Rule of thumb: maximize narrow transformations, minimize wide ones. If you must shuffle, reduce data volume first (filter/select before groupBy)."

---

### Q5: "When would you use Spark vs Pandas? What about Dask or Polars?"

**Answer:**  
"**Pandas**: Data fits in RAM (<10GB). Fastest for small data, richest API.
**Spark**: Data doesn't fit in RAM or needs distributed processing (>10GB). Enterprise standard, integrates with data lakes.
**Dask**: Pandas-like API that scales out. Good bridge between Pandas and Spark — great for Pandas users who need to scale without learning Spark.
**Polars**: Rust-based, very fast single-machine processing. Handles larger-than-RAM data via lazy evaluation and streaming. Growing fast.

My rule: start with Pandas. When it’s too slow, try Polars (same machine). When data is truly distributed, use Spark."

In [ ]:
# Cleanup Spark session
if SPARK_AVAILABLE:
    spark.stop()
    print("✅ Spark session stopped.")

---

## 🎓 Key Takeaways

1. **Spark solves the scale problem** — when data doesn’t fit in memory, distribute it
2. **Driver + Executors** — understand the architecture for debugging
3. **Lazy evaluation + DAG** — Spark optimizes your entire plan before executing
4. **Minimize shuffles** — they’re the #1 performance killer
5. **Cache wisely** — for repeated access to the same data
6. **Parquet + Spark** — the combination enables column pruning and predicate pushdown

---

➡️ **Next Lesson**: [Lesson 7: Building End-to-End Data Pipelines](./lesson_07_end_to_end_pipeline.ipynb)